pip install BeautifulSoup
pip install lxml
pip install requests

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import lxml.html

In [2]:
page_test  = requests.get('http://dilbert.com/')
page_test

<Response [200]>

In [4]:
page_test.status_code

200

Lieferando, static

In [5]:
url = 'https://www.lieferando.de/bewertungen-yoko-sushi-mitte'
page = requests.get(url)

In [6]:
page_bs = bs(page.text, "lxml")
page_h = lxml.html.fromstring(page.text)

In [7]:
#menu-tab-content > div.restaurantreviews.Q07131O
# //*[@id="menu-tab-content"]/div[1]

In [8]:
xpath_review = '//*[contains(@class, "restaurantreview")]'
xpath_review = '//*[@class="restaurantreview"]'
reviews = page_h.xpath(xpath_review)
len(reviews)

20

In [9]:
xpath_tmp =  '//*[@id="menu-tab-content"]/div/*[@class="restaurantreview"]'
page_h.xpath(xpath_tmp)[0].getchildren()[0].tag

'div'

In [10]:
rev = page_bs.find_all(class_="restaurantreview")[0]
len(page_bs.find_all(class_="restaurantreview"))

21

In [11]:
rev

<div class="restaurantreview hidden" id="restaurantreviewJs" itemscope="" itemtype="http://schema.org/Review">
<div class="authoravatar"><img src="/assets/images/profile.jpg"/></div>
<div class="reviewauthor" itemprop="about">
<span itemprop="author">Anonym</span>
</div>
<div class="avatarbar">
<span class="reviewdate" itemprop="datePublished"> Heute um <span class="reviewdate-now"></span></span>
<div class="ratingscontainer">
<div class="review-rating">
            qualität            <div class="review-stars review-stars-quality">
<span class="review-stars-range"></span>
</div>
</div>
<div class="review-rating">
<span class="review-is-delivery">lieferung</span>
<span class="review-is-pickup hidden">abholzeit:</span>
<div class="review-stars review-stars-delivery">
<span class="review-stars-range" style="width: 100%;"></span>
</div>
</div>
</div>
</div>
<div class="reviewbody" itemprop="reviewBody"></div>
<div class="reviewnote-sunday reviewnote hidden">Diese Bestellung wurde an einem

Will have to investigate

In [12]:
for el in rev.find_all('span'):
    print(el, el.get('itemprop') )
    if el.get('itemprop') == 'reviewRating':
        print(el.find('meta').get('content'))

<span itemprop="author">Anonym</span> author
<span class="reviewdate" itemprop="datePublished"> Heute um <span class="reviewdate-now"></span></span> datePublished
<span class="reviewdate-now"></span> None
<span class="review-stars-range"></span> None
<span class="review-is-delivery">lieferung</span> None
<span class="review-is-pickup hidden">abholzeit:</span> None
<span class="review-stars-range" style="width: 100%;"></span> None


In [16]:
lief = requests.get('https://www.lieferheld.de/api/restaurants/38536/comments/?fields=comment&has_text=1&offset=0&limit=30')
len(lief.json()['data'])

4

In [17]:
lief.json()

{'data': [{'comment': {'answer': '',
    'comment_id': '93623695-1573-4e87-9dd1-d4ac768b8aa8',
    'created_at': '2017-07-28T18:10:45Z',
    'downvote_count': 0,
    'message': 'Überragend schnell bei mir. \nEine Speise war lecker aber nich ganz meins, aber das werfe ich niemandem vor, wenn ich was bestelle, was ich nicht kenne ;)',
    'object_id': '1707281841588',
    'object_uri': '',
    'rating': 5,
    'tags': ['marvin', 'review'],
    'upvote_count': 0},
   'id': '93623695-1573-4e87-9dd1-d4ac768b8aa8',
   'name': 'Marcel',
   'uri': '/restaurants/38536/comments/93623695-1573-4e87-9dd1-d4ac768b8aa8',
   'user_id': '',
   'user_uri': ''},
  {'comment': {'answer': '',
    'comment_id': '367924e4-b153-473f-9423-4c1096d1e26e',
    'created_at': '2017-07-16T12:54:46Z',
    'downvote_count': 0,
    'message': 'Super lecker!',
    'object_id': '1707161232516',
    'object_uri': '',
    'rating': 5,
    'tags': ['marvin', 'review'],
    'upvote_count': 0},
   'id': '367924e4-b153-473f-94

In [18]:
el.attrs['style'].split(" ")[-1]

'100%;'

In [1]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-

import csv
import datetime
import json
import argparse
import os
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup as bs
import iso8601


def is_404(content):
    if len(content.find_all(class_="restaurantreview")) == 0:
        return True
    for el in content.find_all('link'):
        if el.has_attr("href"):
            if "lieferando.de/404" in el['href']:
                return True
    return False


def get_review_comment_lieferando(review):
    comment = review.find(class_="reviewbody")
    if comment:
        return comment.text
    else:
        return ""


def get_rating_lieferando(review):
    for el in review.find_all('span'):
        if el.get('itemprop') == 'reviewRating':
            try:
                return float(el.find('meta')['content'])
            except ValueError:
                return 0.0


def get_stars_lieferando(review):
    delivery = 0
    quality = 0
    if len(review.find(class_="ratingscontainer").find_all(class_="review-rating")) == 2:
        for el in review.find(class_="ratingscontainer").find_all(class_="review-rating"):
            value = el.find(class_="review-stars-range").get('style')
            if not value:
                return None, None
            value = value.strip(";%").split(":")[-1]
            value = int(int(value) / 20)
            if "qualität" in el.text or "quality" in el.text:
                quality = value
            else:
                delivery = value
    return delivery, quality


def format_date(day, hour):
    return day + "T" + hour + "Z"


def parse_date_lieferando(content):
    day = content.get('content')
    try:
        _, hour = content.text.split(" um ")
        #         print("yes?")
        return format_date(day, hour)
    except:
        return day


# 2017-03-27T12:46:25Z
def parse_date_other(content):
    day, temp_hour = content.get('content').split("T")
    #     day, temp_hour = content.split("T")
    hour = temp_hour.strip("Z")
    return format_date(day, hour)


def extract_review_lieferando(review_content):
    review = {}
    review['Comment'] = get_review_comment_lieferando(review_content)
    delivery, quality = get_stars_lieferando(review_content)
    if not quality:
        return None
    review['Delivery'] = delivery
    review['Food quality'] = quality
    overall = get_rating_lieferando(review_content)
    # if overall != (delivery + quality) / 2:
    #     print(overall, delivery, quality)
    review['Overall stars'] = overall
    review['Datetime'] = parse_date_lieferando(review_content.find(class_="reviewdate"))
    return review


def extract_reviews_lieferando(content, brand=None):
    for this_one in content.find_all(class_="restaurantreview"):
        review = extract_review_lieferando(this_one)
        if review:
            review['Brand'] = brand
            review["Platform"] = "Lieferando"
            review['id'] = hash_review(review)
            yield review


def create_headers_pizza(url):
    headers_pizza = {'Host': 'pizza.de',
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
                     'Referer': url,
                     'x-mrv-namespace': 'pizza.de'}
    return headers_pizza


def create_headers_held(url, host=""):
    headers_held = {
        'Authentication':'LH api-key=BqFXeTedMu1LQazCYZznkzyL5CFffcWIDW7GEpmCFVAPLi1dA4cdt76BnXkyEuqWAbCf8ZWtADOzaz5851LQj1dlppQVZSxPPAe0cA0g7Tn2GoXWTdfStKk5yrKrrB0J',
        'Connection': 'keep-alive',
        'Host': host,
        'Referer': url,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    return headers_held


def yield_reviews(page_data, brand, platform):
    fields = {'created_at': 'Datetime',
              'message': 'Comment',
              'rating': 'Overall stars'}
    for base_review in page_data['data']:
        review = {"Brand": brand,
                  "Platform": platform}
        for key, value in fields.items():
            review[value] = base_review['comment'][key]
        for el in LIEFERANDO_SPECIFIC:
            review[el] = None
        yield review


def get_reviews_other(url, brand, platform, location):
    counter = 0
    platforms = get_platforms()
    headers = platforms[platform]["headers"](url)
    reviews = []
    restaurant_code = url.split("/")[-2]
    while True:
        data_url = '{}{}/comments/?fields=comment&limit={}&offset={}&sort='.format(platforms[
                                                                                       platform]["base_url"],
                                                                                   restaurant_code, 100, counter*100)
        # page = requests.get(data_url, headers=headers)
        page = requests.get(data_url)
        if page.status_code != 200:
            break
        page_data = page.json()
        if len(page_data['data']) == 0:
            break
        for review in yield_reviews(page_data, brand, platform):
            print(". ", end="", flush=True)
            review['id'] = hash_review(review)
            review['location'] = location
            reviews.append(review)
        counter += 1
    return reviews


def get_reviews_lieferando(brand, original_url, location, pages=None):
    counter = 1
    # original_url = "https://www.lieferando.de/lieferservice-gringo-burritos-berlin/bewertungen"
    url = original_url
    reviews = []
    # for i in range(1):
    while True:
        content = requests.get(url)
        # workaround for rewriting
        if counter == 1 and content.url != original_url:
            print("changing url from {} to {}".format(original_url, content.url))
            original_url = content.url
        bs_content = bs(content.text, "lxml")
        if is_404(bs_content):
            break
        for review in extract_reviews_lieferando(bs_content, brand):
            review['location'] = location
            print(". ", end="", flush=True)
            reviews.append(review)
        counter += 1
        url = original_url + "/" + str(counter)
        if pages and counter > pages:
            break
    print()
    print("{} reviews from {}".format(len(reviews), original_url))
    return reviews


def hash_review(review:dict):
    # return hash("".join([str(r) for r in review.values()]))
    return review['Datetime']+review['Brand']+review['Platform']


def get_lieferando_review_url(url):
    # https: // www.lieferando.de / greengurus
    # https: // www.lieferando.de / bewertungen - greengurus
    p = urlparse(url)
    p_path = "/" + "bewertungen-" + p.path.strip("/")
    p_url = p.scheme + "://" + p.hostname + p_path
    return p_url



ModuleNotFoundError: No module named 'iso8601'

In [ ]:
reviews = get_reviews_lieferando('Sushi','https://www.lieferando.de/bewertungen-yoko-sushi-mitte',None)

In [135]:
head = create_headers_pizza('https://pizza.de/lieferservice/berlin/restaurant-yoko-sushi-berlin-martin-luther-str-410/9647/')
head

{'Host': 'pizza.de',
 'Referer': 'https://pizza.de/lieferservice/berlin/restaurant-yoko-sushi-berlin-martin-luther-str-410/9647/',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
 'x-mrv-namespace': 'pizza.de'}

In [164]:
head = {
# "Host":"pizza.de",
# "Referer":"https://pizza.de/lieferservice/berlin/restaurant-yoko-sushi-berlin-martin-luther-str-410/9647/",
# "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
"x-mrv-namespace":"pizza.de"
}
url_p = 'https://pizza.de/marvin-api/reviews/?businessId=9647&size=5000&order=&'
url_l = 'https://www.lieferheld.de/api/restaurants/10475/comments/?fields=comment&has_text=1&offset=700&limit=700'
# head = {}
req_try = requests.get(url_l, headers=head)
len(req_try.json()['data'])
# req_try.json()

209

In [166]:
trip = requests.get('https://www.tripadvisor.de/ShowUserReviews-g187323-d2694703-r162455177-BBI_Berlin_Burger_International-Berlin.html')
trip

<Response [200]>

In [ ]:
visited = {}
to_visit = {}

In [170]:

page_trip = lxml.html.fromstring(trip.text)
for p in page_trip.xpath('//*[contains(@class, "pageNum")]'):
    print(p.text, p.get('href') )

None None
1 /ShowUserReviews-g187323-d2694703-r524421296-BBI_Berlin_Burger_International-Berlin.html
90 /ShowUserReviews-g187323-d2694703-r176926084-BBI_Berlin_Burger_International-Berlin.html
91 /ShowUserReviews-g187323-d2694703-r162455177-BBI_Berlin_Burger_International-Berlin.html


In [158]:
req_try.json().keys()
req_try.json()['pagination']

{'limit': 700, 'offset': 0, 'page': 0, 'total_items': 909, 'total_pages': 0}

In [165]:
req_try.json()['data']
req_try.json().keys()

dict_keys(['data', 'pagination', 'errors'])

In [13]:
# extract_review_lief(rev)
base_url = 'https://www.lieferando.de/yoko-sushi-mitte'
url1 = get_lieferando_review_url(base_url)
url1

'https://www.lieferando.de/bewertungen-yoko-sushi-mitte'

In [14]:
review_list = list(get_reviews_lieferando("name", url1, None))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
151 reviews from https://www.lieferando.de/bewertungen-yoko-sushi-mitte


In [15]:
counter = 1
brand = "yoko-sushi-mitte"
base_url = "https://www.lieferando.de/bewertungen"
original_url = base_url + brand + "/"
reviews = []
while True:
    url = original_url + "/" + str(counter)
    content = requests.get(url)
    # workaround for rewriting
#     if counter == 1 and content.url != original_url:
#         original_url = content.url
    bs_content = bs(content.text, "lxml")
    if is_404(bs_content):
        break
    for review in extract_reviews_lieferando(bs_content, brand):
        print(". ", end="", flush=True)
        reviews.append(review)
    counter += 1
#     if counter == 2:
#         break
print()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 


In [16]:
len(reviews)

151

Pizza.de, dynamic, 1

In [17]:
original_url = 'https://pizza.de/lieferservice/berlin/restaurant-yoko-sushi-berlin-martin-luther-str-410/9647/'
number = original_url.strip("/").split('/')[-1]
number

'9647'

In [18]:
limit = 100
offset = 900
api_url = "https://pizza.de/dowant-api/restaurants/{}/comments/?fields=comment&limit={}&offset={}&sort=".format(number, limit, offset)
page_pizza = requests.get(api_url)
len(page_pizza.json()['data'])

58

In [19]:
page = requests.get('https://pizza.de/dowant-api/restaurants/9647/comments/?fields=comment&limit=50&offset=0&sort=')
page.json()
page_rv = bs(page.text, "html5lib")
# pattern = '<html><body><p>([\w\W]+)</p></body></html>'
# text = page_rv.json()

In [20]:
import json
res = json.loads(page_pizza.text)
res

{'data': [{'comment': {'answer': '',
    'comment_id': '3661343',
    'created_at': '2011-09-30T11:06:31Z',
    'downvote_count': 0,
    'message': 'super lieferzeit; gutes preis/leistungs-verhältnis; yoko rolls sind irgendwie im durchmesser zu groß; der reis ist leider nicht gewürzt und anstatt aal habe ich hering bekommen...',
    'object_id': '',
    'object_uri': '',
    'rating': 3,
    'tags': ['marvin', 'review'],
    'upvote_count': 0},
   'id': '3661343',
   'name': 'Anonymous',
   'uri': '/restaurants/9647/comments/3661343',
   'user_id': '',
   'user_uri': ''},
  {'comment': {'answer': '',
    'comment_id': '3646661',
    'created_at': '2011-09-29T14:55:16Z',
    'downvote_count': 0,
    'message': 'lecker.',
    'object_id': '',
    'object_uri': '',
    'rating': 5,
    'tags': ['marvin', 'review'],
    'upvote_count': 0},
   'id': '3646661',
   'name': 'Anonymous',
   'uri': '/restaurants/9647/comments/3646661',
   'user_id': '',
   'user_uri': ''},
  {'comment': {'answer

In [21]:
res == page_pizza.json()

True

In [22]:
page_pizza.json()['data'][0]

{'comment': {'answer': '',
  'comment_id': '3661343',
  'created_at': '2011-09-30T11:06:31Z',
  'downvote_count': 0,
  'message': 'super lieferzeit; gutes preis/leistungs-verhältnis; yoko rolls sind irgendwie im durchmesser zu groß; der reis ist leider nicht gewürzt und anstatt aal habe ich hering bekommen...',
  'object_id': '',
  'object_uri': '',
  'rating': 3,
  'tags': ['marvin', 'review'],
  'upvote_count': 0},
 'id': '3661343',
 'name': 'Anonymous',
 'uri': '/restaurants/9647/comments/3661343',
 'user_id': '',
 'user_uri': ''}

In [23]:
review = page_pizza.json()['data'][0]['comment']
review

{'answer': '',
 'comment_id': '3661343',
 'created_at': '2011-09-30T11:06:31Z',
 'downvote_count': 0,
 'message': 'super lieferzeit; gutes preis/leistungs-verhältnis; yoko rolls sind irgendwie im durchmesser zu groß; der reis ist leider nicht gewürzt und anstatt aal habe ich hering bekommen...',
 'object_id': '',
 'object_uri': '',
 'rating': 3,
 'tags': ['marvin', 'review'],
 'upvote_count': 0}

In [115]:
review['message']

'super lieferzeit; gutes preis/leistungs-verhältnis; yoko rolls sind irgendwie im durchmesser zu groß; der reis ist leider nicht gewürzt und anstatt aal habe ich hering bekommen...'

In [25]:
financial = requests.get("https://core-api.barchart.com/v1/options/chain?symbol=SPY&fields=strikePrice%2ClastPrice%2CpercentFromLast%2CbidPrice%2Cmidpoint%2CaskPrice%2CpriceChange%2CpercentChange%2Cvolatility%2Cvolume%2CopenInterest%2CoptionType%2CdaysToExpiration%2CexpirationDate%2CsymbolCode%2CsymbolType&groupBy=optionType&expirationDate=2017-08-04&raw=1&meta=field.shortName%2Cfield.type%2Cfield.description)")

In [26]:
financial.json().keys()

dict_keys(['count', 'total', 'data', 'meta'])

Enter selenium

In [116]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [117]:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

In [118]:
driver.get(url)

In [119]:
driver.back()

In [120]:
driver.get("http://www.python.org")

In [122]:
driver.title

'3. An Informal Introduction to Python — Python 3.6.3rc1 documentation'

In [123]:
driver.close()

In [171]:
driver_fire = webdriver.Firefox()

In [172]:
trip_url = "https://www.tripadvisor.de/ShowUserReviews-g187323-d2694703-r499602079-BBI_Berlin_Burger_International-Berlin.html"
driver_fire.get(trip_url)
driver_fire.title

'Mega Burger in Berliner Ambiente - BBI - Berlin Burger International, Berlin Reisebewertungen - TripAdvisor'

In [35]:
# page_trip = requests.get('https://www.tripadvisor.de/ShowUserReviews-g187323-d2694703-r162455282-BBI_Berlin_Burger_International-Berlin.html#REVIEWS')
page_trip = requests.get('https://www.tripadvisor.de/ShowUserReviews-g187323-d2694703-r162455282-BBI_Berlin_Burger_International-Berlin.html#REVIEWS')

In [36]:
trip_bs = bs(page_trip.text, 'html5lib')


In [37]:
trip_h = lxml.html.fromstring(page_trip.text)

In [38]:
page_h.xpath('//*[contains(@class, "next")]')

[<Element a at 0x10b1ebd68>]

In [173]:
numb = driver_fire.find_element_by_class_name('pageNumbers')
numb

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="58f447f0-c772-ec46-8c08-e47aee527891")>

In [174]:
numb.text

'1\n2\n3\n4\n…\n92'

In [175]:
to_click = numb.find_elements_by_tag_name('a')
[a.text for a in to_click]

['1', '2', '3', '4', '…', '92']

In [178]:
click = to_click[0]

In [179]:
click.click()

In [180]:
driver_fire.back()

In [181]:
driver_fire.find_elements_by_xpath('//*[contains(@class, "reviewSelector")]')

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="59a05c0a-24a2-8343-83ff-37a23f0646a1")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="ff41e737-6e46-3f4d-9943-cdf1e4da14a0")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="8135a241-93b6-bd45-93ca-2a5a317cff9f")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="4e6421c6-5653-0e4e-b267-7437932b6fd4")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="fec17cad-3b9d-e44f-aa9b-31cc7ac88d30", element="f2c5d671-6ca6-d549-aa93-bda896f6c190")>]

In [182]:
def visit_next(driver):
    numb = driver.find_element_by_class_name('pageNumbers')
    links = numb.find_elements_by_tag_name('a')
    to_visit = None
    for link in links:
        if link.text not in visited:
            visited.add(link.text)
            print(link.text)
#             do some magic here
            link.click()
            return len(driver.find_elements_by_xpath('//*[contains(@class, "reviewSelector")]'))
    return 0  

In [184]:
visited = set(['1'])
reviews = 0
driver_fire.get(trip_url)
while True:
    new_reviews = visit_next(driver_fire)
    if not new_reviews:
        print("Done")
        break
    reviews += new_reviews
    print("We now have {} reviews".format(reviews))
    if len(visited) > 1200:
        print("Done")
        break

2
We now have 5 reviews
3
We now have 10 reviews
…
We now have 15 reviews
92
We now have 20 reviews
Done


In [137]:
d = webdriver.Firefox()

In [138]:
d.close()

In [46]:
driver_fire.close()

What if you don't want a browser, and don't want it to CLICK things

In [47]:
driver_hless = webdriver.PhantomJS('phantomjs')
driver_hless.get(trip_url)
driver_hless.title

'Mega Burger in Berliner Ambiente - BBI - Berlin Burger International, Berlin Reisebewertungen - TripAdvisor'

In [48]:
visited = set(['1'])
reviews = 0
driver_hless.get(trip_url)
while True:
    new_reviews = visit_next(driver_hless)
    if not new_reviews:
        print("Done")
        break
    reviews += new_reviews
    print("We now have {} reviews".format(reviews))
    if len(visited) > 4:
        print("Done")
        break

2
We now have 5 reviews
3
We now have 10 reviews
4
We now have 15 reviews
5
We now have 20 reviews
Done


In [49]:
driver_hless.close()

In [50]:
driver_fire = webdriver.Firefox()

In [51]:
google_url = "https://www.google.de/search?q=BBI+-+Berlin+Burger+International&oq=BBI+-+Berlin+Burger+International&aqs=chrome..69i57j69i60l2j0j69i60.455j0j7&sourceid=chrome&ie=UTF-8#lrd=0x47a84fb09074cdcd:0xdbb4649be7a5d4b3,1,"

In [52]:
driver_fire.get(google_url)
driver_fire.title

'BBI - Berlin Burger International - Google-Suche'

In [54]:
driver_fire.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [55]:
driver_fire.find_elements_by_xpath('//*[contains(@class, "_D7k gws-localreviews__google-review")]')
element = driver_fire.find_elements_by_xpath('//*[contains(@class, "_D7k gws-localreviews__google-review")]')[-1]
element

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="f4ef6f05-fdd5-3547-8ab9-3cfb65171994", element="12db4a43-5b90-c349-bad6-8a03496ba136")>

In [56]:
location = element.location_once_scrolled_into_view
location['y']

394

In [57]:
driver_fire.execute_script("window.scrollTo(0, {});".format(location['y']))

In [58]:
element = driver_fire.find_elements_by_xpath('//*[contains(@class, "_D7k gws-localreviews__google-review")]')[-1]
element
location = element.location_once_scrolled_into_view
location['y']

281

In [65]:
driver_fire.execute_script("window.scrollTo(0, {});".format(location['y']))

In [69]:
element = driver_fire.find_elements_by_xpath('//*[contains(@class, "_D7k gws-localreviews__google-review")]')[-1]
location = element.location_once_scrolled_into_view
print(location['y'])
driver_fire.execute_script("window.scrollTo(0, {});".format(location['y']))

398


In [70]:
driver_fire.close()

Mailboxes

In [253]:
import mailbox
import email
import json

MBOX = 'resources/ch06-mailboxes/data/northpole.mbox'

mbox = mailbox.mbox(MBOX)


2

In [256]:
msg = mbox[0]

In [259]:
msg.keys()

['Message-ID',
 'References',
 'In-Reply-To',
 'Date',
 'From',
 'To',
 'Subject',
 'Mime-Version',
 'Content-Type',
 'Content-Transfer-Encoding']

In [260]:
print(msg.get_payload())

Sounds good. See you at the usual location.

Thanks,
-S

 -----Original Message-----
From:   Rudolph  
Sent:   Friday, December 25, 2009 12:04 AM
To: Claus, Santa
Subject:    FWD:  Tonight

Santa -

Running a bit late. Will come grab you shortly. Standby.
    
Rudy

Begin forwarded message:

> Last batch of toys was just loaded onto sleigh. 
>
> Please proceed per the norm.
>
> Regards,
> Buddy
>
> --
> Buddy the Elf
> Chief Elf
> Workshop Operations
> North Pole
> buddy.the.elf@northpole.example.org



In [238]:
body = msg.get_payload()

In [262]:
for line in body.splitlines():
    if not line.startswith(">"):
        if "-----Original Message-----" in line:
            break
        print(line)

Sounds good. See you at the usual location.

Thanks,
-S



In [243]:
dc_url = "https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=ListRecords&from=2001-03-22&until=2001-06-12&set=bmcbioc&metadataPrefix=pmc"
xml_file = requests.get(dc_url)

In [248]:
import xmltodict

In [263]:
xml_dict = xmltodict.parse(xml_file.text)

In [264]:
xml_dict['OAI-PMH'].keys()

odict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', 'responseDate', 'request', 'ListRecords'])

In [270]:
od = xml_dict['OAI-PMH']['ListRecords']
od.keys()

odict_keys(['record'])

In [271]:
od['record']

[OrderedDict([('header',
               OrderedDict([('identifier', 'oai:pubmedcentral.nih.gov:29057'),
                            ('datestamp', '2001-03-22'),
                            ('setSpec', ['bmcbioc', 'pmc-open'])])),
              ('metadata',
               OrderedDict([('article',
                             OrderedDict([('@xmlns:xlink',
                                           'http://www.w3.org/1999/xlink'),
                                          ('@xmlns:mml',
                                           'http://www.w3.org/1998/Math/MathML'),
                                          ('@xmlns:xsi',
                                           'http://www.w3.org/2001/XMLSchema-instance'),
                                          ('@xmlns',
                                           'https://dtd.nlm.nih.gov/ns/archiving/2.3/'),
                                          ('@xsi:schemaLocation',
                                           'https://dtd.nlm.nih.gov/ns/arch